In [26]:
import os
import SimpleITK as sitk
import numpy as np
import pandas as pd

In [15]:
def binary_split_array(array: np.ndarray, labels: list=None) -> np.ndarray:
    """Similar to separate segmentation. Splits array by unique values decoded in binary.
    """
    if labels.size != 0:
        unique_values = labels
    else:
        unique_values = np.unique(array)
    result = np.zeros((len(unique_values), ) + array.shape)
    for i, value in enumerate(unique_values):
        temp = np.array(array)
        temp[temp != value] = 0
        temp[temp == value] = 1
        result[i] = temp
    return result

https://en.wikipedia.org/wiki/S%C3%B8rensen%E2%80%93Dice_coefficient

In [21]:
def calculate_separate_dice(x: np.ndarray, y: np.ndarray, labels: np.ndarray=None) -> np.ndarray:
    """Calculate the Sørensen–Dice coefficient for two numpy arrays x and y 
    where each label is calculated separately. The inputs should match in size/ dimensions."""
    x = binary_split_array(x+1, labels+1)
    y = binary_split_array(y+1, labels+1)
    axis = tuple(np.arange(1, x.ndim))
    return 2*np.sum(x * y, axis=axis) \
            / (np.sum(x, axis=axis) + np.sum(y, axis=axis))

In [22]:
def calculate_overall_dice(x: np.ndarray, y: np.ndarray) -> np.ndarray:
    """Calculate the Sørensen–Dice coefficient for two numpy arrays x and y.
    The inputs should match in size/ dimensions."""
    return 2.0 * np.sum(y[y==x]) / (np.sum(x) + np.sum(y))

In [8]:
labels = {
    0: "background",
    1: "cortical_gray_matter",
    2: "basal_ganglia",
    3: "white_matter",
    4: "white_matter_lesions",
    5: "cerebrospinal_fluid_in_the_extracerebral_space",
    6: "ventricles",
    7: "cerebellum",
    8: "brain_stem",
    9: "Infarction",
    10: "Other"
}

In [62]:
columns = ["filename", "overall"]+list(labels.values())
df = pd.DataFrame(columns=columns)

In [73]:
base_path ="./1/"
truth = sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(base_path, "ground_truth.nii.gz")))

dfs = []

for filename in os.listdir(base_path):
    data = sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(base_path, filename)))
    overall = calculate_overall_dice(truth, data)
    scores = calculate_separate_dice(truth, data, np.array(list(labels.keys())))
    dfs.append(pd.DataFrame([[filename, overall] + list(scores)], columns=columns))
    
df = pd.concat(dfs)
df = df.fillna(0)

<ipython-input-21-077947e97284>:7: RuntimeWarning: invalid value encountered in true_divide
  return 2*np.sum(x * y, axis=axis) \


In [74]:
df.head(10)

,filename,overall,background,cortical_gray_matter,basal_ganglia,white_matter,white_matter_lesions,cerebrospinal_fluid_in_the_extracerebral_space,ventricles,cerebellum,brain_stem,Infarction,Other
0,ground_truth.nii.gz,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,0.0
0,T1_IR_segmentation.nii.gz,0.837894,0.995315,0.778894,0.753260,0.850016,0.008391,0.805274,0.891454,0.923390,0.921437,0.0,0.0
0,all_segmentation.nii.gz,0.879781,0.996677,0.851393,0.859805,0.899125,0.707377,0.842879,0.954240,0.936121,0.947985,0.0,0.0
0,FLAIR_segmentation.nii.gz,0.007505,0.902133,0.007006,0.085024,0.005225,0.065534,0.003832,0.001031,0.007226,0.009544,0.0,0.0
0,FLAIR_IR_segmentation.nii.gz,0.015563,0.900293,0.001567,0.000000,0.000000,0.002136,0.010340,0.000000,0.066886,0.000507,0.0,0.0
0,IR_segmentation.nii.gz,0.000000,0.899740,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
0,FLAIR_T1_segmentation.nii.gz,0.802251,0.988953,0.830728,0.784743,0.889113,0.677129,0.709077,0.487705,0.927783,0.913884,0.0,0.0
0,T1_segmentation.nii.gz,0.672664,0.979190,0.662121,0.449680,0.820655,0.000000,0.540121,0.187031,0.815463,0.848116,0.0,0.0


In [69]:
df.head(10)

,filename,overall,background,cortical_gray_matter,basal_ganglia,white_matter,white_matter_lesions,cerebrospinal_fluid_in_the_extracerebral_space,ventricles,cerebellum,brain_stem,Infarction,Other
0,ground_truth.nii.gz,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0
0,T1_IR_segmentation.nii.gz,0.855630,0.994299,0.795566,0.768058,0.841498,0.010008,0.865256,0.834332,0.947693,0.953292,0.439646,0.0
0,all_segmentation.nii.gz,0.952196,0.995641,0.936953,0.976348,0.959054,0.974941,0.927357,0.993867,0.990550,0.993440,0.934871,0.0
0,FLAIR_segmentation.nii.gz,0.010967,0.884772,0.008450,0.070078,0.013476,0.211260,0.003256,0.001186,0.002280,0.000000,0.000000,0.0
0,FLAIR_IR_segmentation.nii.gz,0.030809,0.884055,0.006827,0.000000,0.000030,0.000304,0.026225,0.000000,0.119410,0.008967,0.000000,0.0
0,IR_segmentation.nii.gz,0.000000,0.882211,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
0,FLAIR_T1_segmentation.nii.gz,0.810990,0.978870,0.864658,0.780747,0.888772,0.789838,0.761399,0.330035,0.962816,0.940254,0.725541,0.0
0,T1_segmentation.nii.gz,0.642126,0.967635,0.695763,0.390327,0.801724,0.002656,0.582945,0.099579,0.729676,0.735713,0.000000,0.0
